In [ ]:
#julia

In [ ]:
#Haakon
import lightgbm as lgb  # standard alias
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy.stats import norm

train = pd.read_csv('data/stores_train.csv')
test = pd.read_csv('data/stores_test.csv')

# To see all the columns in output this can be done.
pd.options.display.max_columns=None
train.head()

#I drop a shit tonn of lables because they were too difficult to integrate into our model with dummies
train = train.drop("year", axis='columns')
train = train.drop("store_name", axis='columns')
train = train.drop("plaace_hierarchy_id", axis='columns')
train = train.drop("grunnkrets_id", axis='columns')
train = train.drop("address", axis='columns')
train = train.drop("store_id", axis='columns')
train = pd.get_dummies(train, columns=['sales_channel_name'])
train = pd.get_dummies(train, columns=['chain_name'])
train = pd.get_dummies(train, columns=['mall_name'])


In [ ]:
print("Shape train: %s, test: %s" % (train.shape, test.shape)) #print number of rows and cols of each dataset

pd.options.display.max_columns = None # Show all cols
train.describe() # shows metadata of the data


""" # plot of data in a normalized curve
fig, ax = plt.subplots(1,2)
width, height = fig.get_size_inches()
fig.set_size_inches(width*2, height)
sns.distplot(train['revenue'], ax=ax[0], fit=norm)
sns.distplot(np.log(train[('revenue')]+1), ax=ax[1], fit= norm) """

In [ ]:
#lgbm things
from sklearn.model_selection import train_test_split
y = train["revenue"]
y.sample(3)

#split training and test data
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)
#Hyper parameters
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l1','l2'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000
}

#fit the model takes some time
gbm = lgb.LGBMRegressor(**hyper_params)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=)

In [ ]:

import math
#predict
y_pred = gbm.predict(X_train, num_iteration=gbm.best_iteration_)

#calc rmse
MSE = np.square(np.subtract(y_train,y_pred)).mean() 
RMSE = math.sqrt(MSE)
print(RMSE)